In [ ]:
# JP 2024
import pandas as pd
import requests
import plotly.graph_objects as go

# Fetch the data
df = pd.read_csv(
    'https://ourworldindata.org/grapher/gini-coefficient-vs-gdp-per-capita-pip.csv?v=1&csvType=full&useColumnShortNames=true',
    storage_options = {'User-Agent': 'Mozilla/5.0'}
)
df = df.rename(columns = {
    'Year': 'year',
    'ny_gdp_pcap_pp_kd': 'gdp',
    'Entity': 'country'    
    }
)

cols = ['country', 'year', 'gdp', 'gini']
df = df.dropna(subset = cols)
df = df[cols]
df = df.sort_values(by = cols)
df['gdp'] = df['gdp'] / 1000

# Create a figure
fig = go.Figure()

# Add traces for each country
countries = df['country'].unique()
for country in countries:
    data = df[df['country'] == country]
    
    fig.add_trace(go.Scatter(
        x = data['gdp'],
        y = data['gini'],
        mode = 'lines+markers+text',
        line = dict(shape = 'spline', smoothing = 0.1),
        marker = dict(size = 12,  symbol= 'arrow-bar-up', angleref = 'previous'),
        name = country,
        hovertemplate = 'Year: %{text}<br>gdp: %{x:.1f} k$<br>Gini: %{y:.2f}',
        text = data['year'],
        textposition = 'top center'
    )
)

# Update layout
fig.update_layout(
    width = 1400, height = 700,
    xaxis_title = 'GDP per Capita',
    yaxis_title = 'Gini Index'
)

fig.for_each_trace(lambda trace: trace.update(visible = 'legendonly') if trace.name not in ['Portugal', 'United States'] else ())

# Function to update the title based on visible traces
def update_title(fig):
    visible_traces = [trace.name for trace in fig.data if trace.visible is True or trace.visible == None]
    fig.update_layout(title='Selected Countries: ' + ', '.join(visible_traces))

# Initial title update
update_title(fig)

# Show the plot
fig.show()